# Command

The Command pattern is a behavioral design pattern that turns a request into a stand-alone object that contains all information about the request. This transformation allows for parameterizing methods with different requests, queuing requests, and logging the requests. It also provides support for undoable operations.

In [2]:
# Command Interface
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self):
        pass
    
    @abstractmethod
    def undo(self):
        pass

In [3]:
# Concrete Command
 
class InsertTextCommand(Command):
    def __init__(self, document, text, position):
        self.document = document
        self.text = text
        self.position = position

    def execute(self):
        self.document.insert_text(self.text, self.position)

    def undo(self):
        self.document.delete_text(self.position, len(self.text))

class DeleteTextCommand(Command):
    def __init__(self, document, position, length):
        self.document = document
        self.position = position
        self.length = length
        self.deleted_text = ""

    def execute(self):
        self.deleted_text = self.document.delete_text(self.position, self.length)

    def undo(self):
        self.document.insert_text(self.deleted_text, self.position)

In [4]:
# Receiver
class Document:
    def __init__(self):
        self.content = ""

    def insert_text(self, text, position):
        self.content = self.content[:position] + text + self.content[position:]

    def delete_text(self, position, length):
        deleted_text = self.content[position:position + length]
        self.content = self.content[:position] + self.content[position + length:]
        return deleted_text

    def __str__(self):
        return self.content

In [5]:
# Invoker
class TextEditor:
    def __init__(self):
        self.document = Document()
        self.history = []
        self.undo_stack = []

    def execute_command(self, command):
        command.execute()
        self.history.append(command)
        self.undo_stack.clear()  # Clear the redo stack

    def undo(self):
        if not self.history:
            return
        command = self.history.pop()
        command.undo()
        self.undo_stack.append(command)

    def redo(self):
        if not self.undo_stack:
            return
        command = self.undo_stack.pop()
        command.execute()
        self.history.append(command)

    def show_document(self):
        print(self.document)

In [6]:
# Client
 
editor = TextEditor()

In [7]:
def editor_stacks():
    print("Undo:", editor.undo_stack)
    print("History:", editor.history)

In [8]:
# Insert "Hello" at position 0
editor.execute_command(InsertTextCommand(editor.document, "Hello", 0))
editor.show_document()  # Output: Hello
editor_stacks()

# Insert " World" at position 5
editor.execute_command(InsertTextCommand(editor.document, " World", 5))
editor.show_document()  # Output: Hello World
editor_stacks()

# Undo last command (deletes " World")
editor.undo()
editor.show_document()  # Output: Hello
editor_stacks()

# Redo last undone command (re-inserts " World")
editor.redo()
editor.show_document()  # Output: Hello World
editor_stacks()

# Delete " World" from position 5
editor.execute_command(DeleteTextCommand(editor.document, 5, 6))
editor.show_document()  # Output: Hello
editor_stacks()

# Undo last command (re-inserts " World")
editor.undo()
editor.show_document()  # Output: Hello World
editor_stacks()


Hello
Undo: []
History: [<__main__.InsertTextCommand object at 0x000001FDA3402D50>]
Hello World
Undo: []
History: [<__main__.InsertTextCommand object at 0x000001FDA3402D50>, <__main__.InsertTextCommand object at 0x000001FDA32D1130>]
Hello
Undo: [<__main__.InsertTextCommand object at 0x000001FDA32D1130>]
History: [<__main__.InsertTextCommand object at 0x000001FDA3402D50>]
Hello World
Undo: []
History: [<__main__.InsertTextCommand object at 0x000001FDA3402D50>, <__main__.InsertTextCommand object at 0x000001FDA32D1130>]
Hello
Undo: []
History: [<__main__.InsertTextCommand object at 0x000001FDA3402D50>, <__main__.InsertTextCommand object at 0x000001FDA32D1130>, <__main__.DeleteTextCommand object at 0x000001FDA3403FB0>]
Hello World
Undo: [<__main__.DeleteTextCommand object at 0x000001FDA3403FB0>]
History: [<__main__.InsertTextCommand object at 0x000001FDA3402D50>, <__main__.InsertTextCommand object at 0x000001FDA32D1130>]
